In [1]:
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import contextily as cx
import imageio as iio
from io import BytesIO
import numpy as np
import matplotlib
from matplotlib.lines import Line2D
matplotlib.use('agg')

In [2]:
gdf = gpd.read_file("./isochrones.geojson")
gdf = gdf.set_crs("EPSG:4326")
points = gpd.read_file("./points.geojson").set_crs("EPSG:4326").to_crs("EPSG:3857")

In [3]:
gdf["time"] = pd.to_datetime(gdf["time"])

In [4]:
COLORS = [
    '#b5152b',
    '#ff9900',
    '#40b6b8',
    '#ff33cc',
    '#417900',
    '#8A2BE2'
]
SOURCE = cx.providers.CartoDB.Positron
DPI = 50
BBOX = gdf.geometry.total_bounds
IMG, EXT = cx.bounds2img(*BBOX, ll=True, source=SOURCE)
FIGSIZE = [r*2 for r in (16,12)]
# PLOT_KWDS = {"facecolor": "#ff990080", "edgecolor":"#ff9900"}
PLOT_KWDS = [
    {"facecolor": "#0070FF66", "edgecolor": "#0070FF"},
    {"facecolor": "#FFF88066", "edgecolor": "#FFF880"},
    {"facecolor": "#E6002666", "edgecolor": "#E60026"},
]

In [14]:
for ix, (name, group) in enumerate(gdf.to_crs("EPSG:3857").groupby("time")):
    fig, ax = plt.subplots(1,1, figsize=FIGSIZE)
    _ = ax.imshow(IMG, extent=EXT)
    _ = ax.axis("off")
    for n, inner_group in reversed(list(group.groupby("interval_id"))):
        plot_kwds = PLOT_KWDS[n]
        if n == 0:
            inner_group.plot(ax=ax, **plot_kwds)
        elif n == 1:
            geom = group.query("interval_id == 1").unary_union.difference(group.query("interval_id == 0").unary_union)
            gpd.GeoDataFrame(geometry=[geom], crs="EPSG:3857").plot(ax=ax, **plot_kwds)
        else:
            geom = group.query("interval_id == 2").unary_union.difference(group.query("interval_id == 1").unary_union)
            gpd.GeoDataFrame(geometry=[geom], crs="EPSG:3857").plot(ax=ax, **plot_kwds) #**dict(plot_kwds, **{"edgecolor": "none"}))
            #inner_group.plot(ax=ax, **dict(plot_kwds, **{"facecolor": "none"}))
    custom_lines = [Line2D([0], [0], color=c["facecolor"], lw=10) for c in PLOT_KWDS]
    legend = ax.legend(custom_lines, ["100", "250", "500"], loc=4, prop={'size': 40}, title="Isochrone limits (s)")
    plt.setp(legend.get_title(),fontsize=50)
    points.plot(ax=ax, color="black", markersize=50)
    text = name.strftime("%A, %H:%M")
    props = {"facecolor": "white", "boxstyle": "square"}
    ax.text(EXT[0] + ((EXT[1]-EXT[0])/2)-5000, EXT[2] + 1500, text, fontsize=48, bbox=props)
    fig.tight_layout()
    plt.savefig(f"./img/frame_{ix}.jpg", dpi=DPI)
    plt.close(fig)

In [15]:
with iio.get_writer("./valhalla.mp4", mode="I") as writer:
    #for repetition in range(3):
        for i in range(ix + 1):
            writer.append_data(iio.imread(f"./img/frame_{i}.jpg"))

## Directions

In [5]:
point_locations = gpd.read_file("./directions_locations.geojson")

In [6]:
max_ = []
for file_index in range(5):
    gdf = gpd.read_file(f"./shp/directions_{0}.shp")
    gdf["time"] = pd.to_datetime(gdf["time"])
    gdf["time"] = pd.to_datetime(gdf["time"])
    gdf["duration_minutes"] = gdf["duration"] / 60
    for ix, (name, group) in enumerate(gdf.groupby("time")):
        _ = np.histogram(group["duration_minutes"], bins=20)
        max_.append(max(_[0]))
MAX_Y = max(max_)

In [8]:
ix = 0
for file_index in range(5):
    gdf = gpd.read_file(f"./shp/directions_{file_index}.shp")
    gdf["time"] = pd.to_datetime(gdf["time"])
    gdf["time"] = pd.to_datetime(gdf["time"])
    gdf["duration_minutes"] = gdf["duration"] / 60
    # BBOX = gdf.geometry.total_bounds
    IN_BBOX = [gdf.geometry.total_bounds[0] - 0.15, gdf.geometry.total_bounds[1], gdf.geometry.total_bounds[2] + 0.15, gdf.geometry.total_bounds[3]]
    # BBOX = [n + 0.01 if i in (0,1) else n - 0.01 for i, n in enumerate(IN_BBOX)] # zoom in, preserve ratio
    IMG, EXT = cx.bounds2img(*IN_BBOX, ll=True, source=SOURCE)
    FIGSIZE = [r*2 for r in (16,12)]
    for name, group in gdf.to_crs("EPSG:3857").groupby("time"):
        fig, (ax, hax) = plt.subplots(2,1, figsize=FIGSIZE, gridspec_kw={'height_ratios': [10, 2]})
        _ = ax.imshow(IMG, extent=EXT)
        _ = ax.axis("off")
        group.plot(ax=ax, edgecolor='#40b6b880', linewidth=2)
        point_locations.to_crs("EPSG:3857").plot(ax=ax, color="black", markersize=10, zorder=2)
        point_locations.iloc[:1].to_crs("EPSG:3857").plot(ax=ax, color="#b5152b", markersize=90, zorder=3)
        bins = list(np.arange(0, max(gdf["duration_minutes"]), max(gdf["duration_minutes"]) / 20))
        group.hist("duration_minutes", ax=hax, bins=bins, color="#40b6b890", edgecolor="#40b6b8")
        hax.set_xlim(min(gdf["duration_minutes"]), max(gdf["duration_minutes"]))
        hax.set_ylim(0, MAX_Y)
        #hax.set_title("Route duration histogram", fontsize=35)
        hax.set_title("", fontsize=35)
        hax.set_facecolor('#d1d1d133')
        hax.set_ylabel("Number of routes", size=30)
        hax.set_xlabel("Route duration (minutes)", size=30)
        hax.grid(color='white', linestyle='-', linewidth=2)
        _ = [item.set_fontsize(30) for item in hax.get_xticklabels() + hax.get_yticklabels()]
        text = name.strftime("%A, %H:%M")
        props = {"facecolor": "white", "boxstyle": "square"}
        ax.text(EXT[0] + ((EXT[1]-EXT[0])/2)-5000, EXT[2] + 3000, text, fontsize=42, bbox=props)
        fig.tight_layout()
        plt.savefig(f"./img/dir_frame_{ix}.jpg", dpi=DPI)
        plt.close(fig)
        ix += 1
    plt.close("all")

In [10]:
with iio.get_writer("./valhalla_directions.mp4", mode="I") as writer:
    for repetition in range(3):
        for i in range(ix):
            writer.append_data(iio.imread(f"./img/dir_frame_{i}.jpg"))